## Generalizing PREPARE and SELECT

**Learning outcomes**

* Understand the action of SELECT and PREPARE oracles.

In [8]:
import numpy as np
import pennylane as qml

In the last node, we found a circuit for applying an arbitrary linear combination of unitaries. It pslit naturally into two subroutines, PREPARE, which prepared the state and coefficients, and SELECT which applied the unitaries. These were somewhat messily defined using nested circuit, but it turns there is a clean generalization of PREPARE and SELECT.

For a linear combination of unitaries:
$$\tilde{U} = \sum_{j} \alpha_j U_J $$
which $\alpha_j > 0$, we'll define PREPARE and SELECT as oracle (black boxes) with the following behaviour:
$$SELECT\Ket{j} \Ket{\psi} = \Ket{j}*U_J* \Ket{\psi} $$
$$PREPARE\Ket{0} = \frac{1}{\sqrt{\alpha}}\sum_{j} \sqrt{\alpha_j} \Ket{j}$$

For a linear combination of unitaries:
$$\tilde{U} = \sum_{j} \alpha_j U_J $$
which $\alpha_j > 0$, we'll define PREPARE and SELECT as oracle (black boxes) with the following behaviour:
$$SELECT\Ket{j} \Ket{\psi} = \Ket{j}*U_J* \Ket{\psi} $$
$$PREPARE\Ket{0} = \frac{1}{\sqrt{\alpha}}\sum_{j} \sqrt{\alpha_j} \Ket{j} = \Ket{\alpha}$$
,where $\alpha  = \sum_{j} \alpha_j$, and $\Ket{j}$ is an orthonormal basis for the auxiliary system with initial state $\Ket{0}$

Since we have only specified hot the PREPARE oracle acts on the state $\Ket{0}$, we have the freedom to chooose *any* unitary which perfoms this. It doesn't matter what it does to the stets orthogonal to $\Ket{0}$! A convenient  chhoice is the *Householder transformation*. For a state $\Ket{v}$, this is defined as
$$R_v = I - 2\Ket{v}\Braket{v}$$

**Codercise H.7.1.** Write a fuction for a creating the Householder transformation as a matrix, assuming the inout a normalized state. You will [np.outer](https://numpy.org/doc/stable/reference/generated/numpy.outer.html) helpful.

In [7]:
def householder(state):
    """Create the matrix form of a Householder transformation.

    Args:
        state (array[complex]): A list of amplitudes representing a state.

    Returns:
        array[complex]: The matrix representation of the Householder transformation.
    """
    matrix = np.identity(2) - 2* np.outer(state,np.conjugate(state))
    return matrix
householder([0,1])

array([[ 1.,  0.],
       [ 0., -1.]])

We'll define the PREPARE oracke as the HOuseholder transformation of the state:
$$\Ket{0-\alpha} = \frac{\Ket{0} - \Ket{\alpha}}{\Ket{\Ket{0} - \Ket{\alpha}}}.$$
This requires a little explanation. First, $\Ket{\alpha}$ is a vector of coefficients which has been normalized and can therefore be interpreted as a state. Then $\Ket{0} - \Ket{\alpha}$ is a difference of state, and is normalized by the denominator.

**Codercise H.7.2** (a) Implement the PREPARE oracle using the Householder transformation; this is available from the previous exercise. For simplicity, we work with a sum of $2^k$ unitaries. This means our auxiliary register will have $k$ qubits, and the state $\Ket{0}$ is the all-zero state (usually denoted $\Ket{0}$).

In [11]:
k_bits = 3
zero_vec = np.array([1] + [0]*(2**k_bits - 1))
print(zero_vec)

[1 0 0 0 0 0 0 0]


In [15]:
k_bits = 2
n_bits = 2
all_bits = k_bits + n_bits
aux = range(k_bits)
main = range(k_bits, all_bits)
dev = qml.device("default.qubit", wires=all_bits)

def PREPARE(alpha_list):
    """Create the PREPARE oracle as a matrix.

    Args:
        alpha_list (array[float]): A list of coefficients.

    Returns:
        array[complex]: The matrix representation of the PREPARE routine.
    """
    zero_vec = np.array([1] + [0]*(2**k_bits - 1))

    alpha_list = np.sqrt(alpha_list)/np.linalg.norm(alpha_list)
    numerator = zero_vec - alpha_list
    denominator = np.linalg.norm(numerator)
    state = numerator/denominator

    return householder(state)

(b) The circuit below shows a PREPARE oracle (red), followed by a SELECT oracle (blue), followed by PREPARE+ (red):
![circuit](./images/H.7.2.1.png)
if $\Ket{0}$ is measured on the auxuliary register, this circuit applies the linear combination $\tilde{U}$ to $\Ket(\psi)$. Implement the LCU circuit shown above. SELECT is provided for you.

In [44]:
k_bits = 2
n_bits = 2
all_bits = k_bits + n_bits
aux = range(k_bits)
main = range(k_bits, all_bits)


In [19]:
def SELECT(U_list):
    """Implement the SELECT oracle for 2^k unitaries."""
    for index in range(2**k_bits):
        ctrl_str = np.binary_repr(index, k_bits) # Create binary representation
        qml.ControlledQubitUnitary(U_list[index], control_wires=aux,wires=main,control_values=ctrl_str)

def LCU(alpha_list, U_list):
    """Implement LCU using PREPARE and SELECT oracles for 2^k unitaries.

    Args:
        alpha_list (list[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices, stored as
        complex arrays.
    """
    qml.QubitUnitary(PREPARE(alpha_list), wires=aux)
    SELECT(U_list)
    qml.adjoint(qml.QubitUnitary)(PREPARE(alpha_list), wires=aux)

(c) Finally, consider the combination of unitarie
$$\tilde{U} = \sum_{j} \alpha_j U_j = X \otimes X + \frac{1}{2}Z \otimes Z + \frac{1}{2}X \otimes Z + Z \otimes X = H \otimes \begin{bmatrix}
1/2 & 1 \\
1 & -1/2
\end{bmatrix}$$
Create a circuit which applies this tp $|11>$. Note that
$$H \otimes \begin{bmatrix}
1/2 & 1 \\
1 & -1/2
\end{bmatrix} \Ket{11}  \propto\Ket{00} - \frac{1}{2} \Ket{01} - \Ket{10} + \frac{1}{2}\Ket{11}$$
so the unnormalized amplitudes should be proportional to this.

In [46]:
U_list = [np.kron(qml.PauliX.compute_matrix(), qml.PauliX.compute_matrix()),
          np.kron(qml.PauliZ.compute_matrix(), qml.PauliZ.compute_matrix()),
          np.kron(qml.PauliX.compute_matrix(), qml.PauliZ.compute_matrix()),
          np.kron(qml.PauliZ.compute_matrix(), qml.PauliX.compute_matrix())]
alpha_list = [1, 0.5, 0.5, 1]

@qml.qnode(dev)
def my_circuit():
    """Apply H(X + Z/2) to the state |11> using LCU."""
    qml.broadcast(qml.PauliX, wires=main, pattern="single")
    LCU(alpha_list, U_list)
    return qml.state()

print("The amplitudes on the main register are proportional to", my_circuit()[:4], ".")

ValueError: operands could not be broadcast together with shapes (4,) (2,) 

We know that PREPARE and SELECT are useful for Hamiltonian simulation. BUt it turns out that we can also use them to implement memory on a quantum computer! We imagine that the auxiliary states $\Ket{j}_A$ are memory addresses and the main register stores data $\Ket{D_j}_D$, prepared from some initial state $\Ket{0}_D$ by acting with a unitary, $\Ket{D_j}_D = U_j\Ket{0}_D$ (we've added subscripts for clarity). The quantum computer may wish not only to retrieve individual states from their addresses, but to create an arbitrary superposition of data states. In order to remember which data belongs where, we assume they remain correlated with their addresses, so our task is to take the initial state $\Ket{0}_A \Ket{0}_D$ to a state of the form:
$$\Ket{\beta} = \sum_{j} \beta_j \Ket{j}_A \Ket{D_j}_D$$
for $\beta_j \geqslant 0$, this can be created using the PREPARE and SELECT oracles associated with the LCU $\tilde{U} = \sum_{j}\beta_j^2U_j$. Specifically,
$$\Ket{\beta} = SELECT * PREPARE \Ket{0}_A \Ket{0}_D.$$
It's possible to modify this to allow for *arbitrary* superpositions, but in the last codercise, you can implement the simpler case of positive coefficients.

*Codercise H.7.3* Use the PREPARE and SELECT procedures you defined above to generate superpositions of two-qubit computational basis states,
$$\Ket{\beta} = \beta_00\Ket{00} + \beta_01\Ket{01} + \beta_10\Ket{10} + \beta_11\Ket{11}.$$
Is this correctly producing the maximally entangled state, $\frac{1}{\sqrt{2}}(\Ket{00} + \Ket{11})$?

In [ ]:
@qml.qnode(dev)
def quantum_memory(beta_list):
    """Produce a data state with positive coefficients beta_list.

    Args:
        beta_list (array[float]): The amplitudes for our superposition.

    Returns:
        array[float]: The state on both address and data registers.
    """
    U_list = [np.identity(2), np.identity(2),
              np.identity(2), np.identity(2)] # MODIFY THIS

    return qml.state()

beta_list = [1, 0, 0, 1]
normalized_coefficients = [quantum_memory(beta_list)[i].item() for i in range(0, 20, 5)]
print("The amplitudes on the main register are proportional to", normalized_coefficients, ".")
